In [1]:
dia = ''

In [2]:
# Parameters
dia = "2024-03-12"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
from scripts.AtualizaUltimosJogos import *
import warnings
warnings.filterwarnings('ignore')

# Pegando os jogos do dia para atualizar
jogos_passados = pd.read_csv(f'jogos_do_dia/{dia}.csv')
dia_jogos = jogos_passados['Date'].iloc[0]
id_jogos = jogos_passados['Fixture ID'].unique().tolist()
print(f'Jogos do dia {dia_jogos}\n')

# Buscando os resultados
results = atualiza_ultimos_jogos(id_jogos = id_jogos)

# Merjando os resultados no dataset de jogos do dia
df_resultados = pd.merge(left=jogos_passados, right=results[['Fixture ID', 'Home_Pts', 'Away_Pts', 'Status']], on='Fixture ID', how='left')
df_resultados['Status'].replace(np.nan, 'Não iniciado', inplace=True)

# Organizando as colunas do df_resultados
df_resultados = df_resultados[['Fixture ID', 'Date', 'League', 'Time', 'Home', 'Away', 'Home_Pts', 'Away_Pts', 'Odds_H',
       'Odds_A', 'Over_Line', 'Odds_Over', 'Odds_Under', 'HA_Line',
       'HA_Odds_H', 'HA_Odds_A', 'Status']]
df_resultados = df_resultados.drop_duplicates(subset=['Date', 'Home', 'Away'], keep='first')
df_resultados.sort_values('Time', inplace=True)
df_resultados.reset_index(inplace=True, drop=True)
# Criando df com jogos que ainda não terminaram
df_restantes = df_resultados[(df_resultados['Status'] != 'Após Tempo Extra') & (df_resultados['Status'] != 'Encerrado') & (df_resultados['Status'] != 'Após Pênaltis')]
df_restantes = df_restantes[['Date', 'League', 'Time', 'Home', 'Away', 'Home_Pts', 'Away_Pts', 'Status']]

# Limpando o df_resultados
df_resultados = df_resultados[(df_resultados['Status'] == 'Após Tempo Extra') | (df_resultados['Status'] == 'Encerrado') | (df_resultados['Status'] == 'Após Pênaltis')]
df_resultados = df_resultados[['Date', 'League', 'Time', 'Home', 'Away', 'Home_Pts', 'Away_Pts', 'Odds_H',
       'Odds_A', 'Over_Line', 'Odds_Over', 'Odds_Under', 'HA_Line',
       'HA_Odds_H', 'HA_Odds_A']]

# Exclui os registros que já existem no dataset da temporada
temporada = pd.read_csv('data/Entire_Season.csv')
n_antes = len(temporada)
temporada = pd.concat([temporada, df_resultados], ignore_index=True)
temporada = temporada.drop_duplicates(subset=['Date', 'Home', 'Away'], keep='first')
temporada.reset_index(drop=True, inplace=True)
temporada.to_csv('data/Entire_Season.csv', index=False)

print(f'Dataset de temporada atualizado com {len(temporada) - n_antes} jogos.\n')
print('----------------------------------------------------------\n')
print(f'Atenção! {len(df_restantes)} jogos não finalizados.\n')
display(df_restantes)

Jogos do dia 2024-03-12



The msedgedriver version (121.0.2277.128) detected in PATH at C:\Users\johnn\Documents\venvs\basketball_models\msedgedriver.exe might not be compatible with the detected MicrosoftEdge version (122.0.2365.66); currently, msedgedriver 122.0.2365.80 is recommended for MicrosoftEdge 122.*, so it is advised to delete the driver in PATH and retry


  0%|                                                                                           | 0/71 [00:00<?, ?it/s]

  1%|█▏                                                                                 | 1/71 [00:02<02:38,  2.26s/it]

  3%|██▎                                                                                | 2/71 [00:03<01:47,  1.56s/it]

...

 99%|████████████████████████████████████████████████████████████████████████████████▊ | 70/71 [01:53<00:01,  1.75s/it]

100%|██████████████████████████████████████████████████████████████████████████████████| 71/71 [01:56<00:00,  1.97s/it]

100%|██████████████████████████████████████████████████████████████████████████████████| 71/71 [01:56<00:00,  1.64s/it]

Dataset de temporada atualizado com 69 jogos.

----------------------------------------------------------

Atenção! 2 jogos não finalizados.



,Date,League,Time,Home,Away,Home_Pts,Away_Pts,Status
12,2024-03-12,África Bal,11:00,Dynamo,Petro,0.0,20.0,Atribuído
58,2024-03-12,Argentina Liga Feminina,21:00,Berazategui F,Obras F,NaN,NaN,Não iniciado
